In [1]:
import numpy as np
import pandas as pd

import methods.model as model
import methods.dists as dists
from batch_runner import run_simulation, generate_activity_attractivity

# Fixed random generator for reproducibility
rng = np.random.default_rng(seed=42)


In [2]:
params = {
    "s": 5,
    "spending_rate": (
        "uniform", 
        [0, 1], 
        lambda N, rng: rng.uniform(1e-16, 1, N)  # ← now accepts rng
    ),
    "initial_balance": (
        "constant", 
        [1000], 
        lambda N, rng: 1000 * np.ones(N)         # ← also accepts rng
    ),
    "decimals": 3,
    "copula": ("joe", 3.15, False),
    "activity_distribution": (
        "powlaw", 
        [1.85, 1, 1838], 
        lambda unif: dists.powlaw_ppf(1.85, 1, 1838)(unif)
    ),
    "attractivity_distribution": (
        "powlaw", 
        [1.87, 1, 2118], 
        lambda unif: dists.powlaw_ppf(1.87, 1, 2118)(unif)
    ),
    "N": 500,
    "T": 50,
    "D": 50 / 43706315,
    "SIZE_SCALE": 1,
    "LENGTH_SCALE": 6,
    "MEAN_IET": 43706315,
    "burstiness": 0.5,
}


test if simulations with fixed seed are identical

In [3]:
saved = 20

transactions_df, metadata, execution_time, nodes_df = run_simulation(params, saved=saved, rng=rng)

print(f"Simulation completed in {execution_time:.2f} seconds")

# Display sample results
display(transactions_df.head())
display(nodes_df.head())
print(metadata)


Simulation completed in 0.05 seconds


,timestamp,source,target,amount,source_bal,target_bal
0,11895.335294,322,305,152.847,847.153,573.072
1,11909.448372,216,118,15.423,984.577,1015.423
2,12069.495583,271,483,26.291,973.709,636.727
3,13029.820456,35,261,218.293,781.707,1218.293
4,14727.576989,28,248,142.709,768.226,1142.709


,index,act_pot,att_pot,act,att,spr,iet
0,0,1.562782,1.077460,3.575643e-08,0.000228,0.773956,"(0.5, 0.5)"
1,1,28.577070,26.824492,6.538430e-07,0.005673,0.438878,"(0.5, 0.5)"
2,2,3.152324,6.670279,7.212514e-08,0.001411,0.858598,"(0.5, 0.5)"
3,3,3.736399,3.130062,8.548877e-08,0.000662,0.697368,"(0.5, 0.5)"
4,4,1.537638,1.819288,3.518115e-08,0.000385,0.094177,"(0.5, 0.5)"


{'sprate_type': 'uniform', 'sprate_params': [0, 1], 'inbal_type': 'constant', 'inbal_params': [1000], 'activity_type': 'powlaw', 'activity_params': [1.85, 1, 1838], 'attractivity_type': 'powlaw', 'attractivity_params': [1.87, 1, 2118], 'N': 500, 'T': 50, 'D': 1.1439994426434715e-06, 's': 5, 'decimals': 3, 'SIZE_SCALE': 1, 'LENGTH_SCALE': 6, 'MEAN_IET': 43706315, 'burstiness': 0.5, 'copula_type': 'joe', 'copula_param': 3.15}


test what is fixed and what not

In [4]:
# Set fixed rng
rng = np.random.default_rng(seed=42)

# Estrai parametri dal dizionario
copula_type, copula_param, reversed_copula = params["copula"]
activity_type, activity_params, activity_generator = params["activity_distribution"]
attractivity_type, attractivity_params, attractivity_generator = params["attractivity_distribution"]
sprate_type, sprate_params, sprate_generator = params["spending_rate"]
inbal_type, inbal_params, inbal_generator = params["initial_balance"]
N = params["N"]

# Genera campioni uniformi tramite la stessa funzione che usi nella simulazione
from methods.dists import paired_samples

unif_act, unif_att = paired_samples(
    N=N,
    params={'copula': copula_type, 'theta': copula_param, 'reversed': reversed_copula},
    rng=rng
)

# Applica le trasformazioni
vect_act = activity_generator(unif_act)
vect_att = attractivity_generator(unif_att)
spending = sprate_generator(N, rng=rng)
initial_bal = inbal_generator(N, rng=rng)

# Calcola hash di ciascun vettore
import hashlib

def hash_array(arr):
    return hashlib.md5(np.array(arr).tobytes()).hexdigest()

print("activity hash:     ", hash_array(vect_act))
print("attractivity hash: ", hash_array(vect_att))
print("spending hash:     ", hash_array(spending))
print("initial_bal hash:  ", hash_array(initial_bal))


TypeError: paired_samples() got an unexpected keyword argument 'rng'

In [5]:
import pycop.simulation as cop
np.random.seed(42)

a1, a2 = cop.simu_archimedean("joe", 2, 10, theta=3.15)
np.random.seed(42)
b1, b2 = cop.simu_archimedean("joe", 2, 10, theta=3.15)

print(np.allclose(a1, b1), np.allclose(a2, b2))


True True


In [6]:
# Test diretto con fixed np.random.seed
np.random.seed(42)
u1, u2 = cop.simu_archimedean("joe", 2, 10, theta=3.15)
print("Sample 1:", u1[:3], u2[:3])

np.random.seed(42)
u1b, u2b = cop.simu_archimedean("joe", 2, 10, theta=3.15)
print("Sample 2:", u1b[:3], u2b[:3])


Sample 1: [0.0065812  0.97839613 0.79017566] [0.25950411 0.91892097 0.61855806]
Sample 2: [0.0065812  0.97839613 0.79017566] [0.25950411 0.91892097 0.61855806]


Proviamo a patchare pycop

In [7]:
import numpy as np
import pycop.simulation as cop

# Create controlled RNG
rng = np.random.default_rng(seed=42)

# Backup original np.random.random
_original_random = np.random.random

# Patch np.random.random to redirect to our rng
np.random.random = rng.random

# Call pycop
a1, a2 = cop.simu_archimedean("joe", 2, 10, theta=3.15)

# Restore original function
np.random.random = _original_random

# Repeat identically
rng = np.random.default_rng(seed=42)
np.random.random = rng.random
b1, b2 = cop.simu_archimedean("joe", 2, 10, theta=3.15)
np.random.random = _original_random

# Check equality
print(np.allclose(a1, b1), np.allclose(a2, b2))


False False


In [9]:
import numpy as np
import pandas as pd
import hashlib
from batch_runner import run_simulation  # replace with your actual path
from methods.dists import paired_samples, random_unifs

# Define test parameters
params = {
    "s": 5,
    "spending_rate": (
        "uniform", 
        [0, 1], 
        lambda N,rng: rng.uniform(1e-16, 1, N)  # ← only np.random
    ),
    "initial_balance": (
        "constant", 
        [1000], 
        lambda N,rng: 1000 * np.ones(N)
    ),
    "decimals": 3,
    "copula": ("joe", 3.15, False),
    "activity_distribution": (
        "powlaw", 
        [1.85, 1, 1838], 
        lambda unif: dists.powlaw_ppf(1.85, 1, 1838)(unif)
    ),
    "attractivity_distribution": (
        "powlaw", 
        [1.87, 1, 2118], 
        lambda unif: dists.powlaw_ppf(1.87, 1, 2118)(unif)
    ),
    "N": 500,
    "T": 50,
    "D": 50 / 43706315,
    "SIZE_SCALE": 1,
    "LENGTH_SCALE": 6,
    "MEAN_IET": 43706315,
    "burstiness": 0.5,
}

# Utility to hash a DataFrame
def hash_df(df):
    return hashlib.md5(df.to_csv(index=False).encode()).hexdigest()

# Run 1
np.random.seed(42)
df1, meta1, _, nodes1 = run_simulation(params, saved=20)
hash1 = hash_df(nodes1)

# Run 2
np.random.seed(42)
df2, meta2, _, nodes2 = run_simulation(params, saved=20)
hash2 = hash_df(nodes2)

# Check if both are identical
print("Run 1 hash:", hash1)
print("Run 2 hash:", hash2)
print("Identical nodes:", hash1 == hash2)


Run 1 hash: 9aeb1c66b88ccd4251478aeb972a926c
Run 2 hash: 9aeb1c66b88ccd4251478aeb972a926c
Identical nodes: True


In [ ]:
t = 50
s = 10
alist = [None]*s
blist = [None]*s
bp = t-s
# j = 0

val = [t for t in range(t)]
for i in range(t):
    if i>=bp:
        # print(val[i])
        alist[t-i-1] = val[i]
        print(alist)
        blist[i-bp] = val[i]
        print(blist)
        # j+=1
    
print(alist)
    
print(blist)


In [ ]:
import pycop.simulation as cop
import numpy as np

np.random.seed(42)

a1, a2 = cop.simu_archimedean("clayton", 2, 10, theta=3)
np.random.seed(42)
b1, b2 = cop.simu_archimedean("clayton", 2, 10, theta=3)

print(np.allclose(a1, b1), np.allclose(a2, b2))


In [ ]:
import numpy as np
import pycop.simulation as cop

# Create our own Generator
rng = np.random.default_rng(seed=42)

# Save original np.random.random
_original_random = np.random.random

# Patch np.random.random with rng.random
np.random.random = rng.random

# Run pycop safely
u1, u2 = cop.simu_archimedean("clayton", 2, 10, theta=3)

# Restore original function immediately after
np.random.random = _original_random
